In [15]:
import os

# Define target directory
TARGET_DIR = os.path.join(".", "output")

# Load the data

import json

path_to_results = '../study/rawresults/test_Results.json'

with open(path_to_results, "r") as jsonRes:
    res = json.load(jsonRes)
    jsonRes.close()

    # Remove answers after 1646089200000

    for ans in res:
        if ans['timestamp'] > 1646089200000:
            print("Removing Entry with ID {}".format(ans['ID']))
            res.remove(ans)


In [16]:
def answersToAnswerTime(answers):
    time = [(v["timestamp"], v) for v in answers]
    time.sort()
    answerTimes = {}
    answerTimes[0] = time[0][1]
    for i in range(1, len(time)):
        answerTimes[(time[i][0] - time[i-1][0]) / 1000] = time[i][1]
    return answerTimes


Compute Attempts, that were answered in less than 60 seconds


In [17]:


from collections import defaultdict


MAX_ATTEMPT = 6
MIN_TIME_PER_ATTEMPT = 60

# Group Data by participantId id
userid_2_answers = defaultdict(list)
for ans in res:
    userid_2_answers[ans["participantId"]].append(ans)


ans_removed_cause_incomplete = 0
att_removed_cause_incomplete = 0

ans_removed_cause_fast = 0
att_removed_cause_fast = 0

toRemove = []


for userid, answers in userid_2_answers.items():
    for i in range(1, MAX_ATTEMPT+1, 1):
        answers_for_attempt = [x for x in answers if x["attempt"] == i]
        if (len(answers_for_attempt) > 0):
            # Not completed
            time = answersToAnswerTime(answers_for_attempt)
            if len([x for x in time.keys()]) < 8:
                ans_removed_cause_incomplete += len([v["ID"]
                                                    for k, v in time.items()])
                att_removed_cause_incomplete += 1
                toRemove.extend(time.values())
                print("User {},\t Attempt: {} \t {}".format(answers_for_attempt[0]["participantId"], i,
                                                            [v["ID"] for k, v in time.items()]))

            elif sum([x for x in time.keys()]) < MIN_TIME_PER_ATTEMPT:
                # To fast
                ans_removed_cause_fast += len([v["ID"]
                                              for k, v in time.items()])
                att_removed_cause_fast += 1
                toRemove.extend(time.values())
                print("User {},\t Attempt: {} \t {}".format(answers_for_attempt[0]["participantId"], i,
                                                            [v["ID"]
                                                                for k, v in time.items()]
                                                            ))


User 4,	 Attempt: 2 	 [18, 19]
User 18,	 Attempt: 1 	 [60, 61, 62]
User 19,	 Attempt: 1 	 [70, 71, 73, 75, 78]
User 21,	 Attempt: 5 	 [221, 222, 223, 224, 225, 226, 227, 228]
User 21,	 Attempt: 6 	 [229, 230, 231, 232, 233, 234, 235, 236]
User 43,	 Attempt: 1 	 [139]
User 42,	 Attempt: 1 	 [143, 148, 150, 152, 153, 154, 155]
User 49,	 Attempt: 1 	 [165]
User 47,	 Attempt: 1 	 [166, 167, 168, 169, 171, 172, 173]
User 51,	 Attempt: 1 	 [188, 189, 190, 191, 192, 193, 194, 195]
User 59,	 Attempt: 1 	 [238, 239, 241, 243, 245]
User 95,	 Attempt: 2 	 [331, 332, 333, 334, 335, 336, 337, 338]
User 102,	 Attempt: 1 	 [363, 364, 365, 366]
User 134,	 Attempt: 1 	 [453, 455]
User 139,	 Attempt: 1 	 [459, 461]
User 140,	 Attempt: 1 	 [462]
User 141,	 Attempt: 1 	 [468]
User 143,	 Attempt: 1 	 [469]
User 167,	 Attempt: 1 	 [526]
User 188,	 Attempt: 2 	 [631]
User 189,	 Attempt: 1 	 [632, 633, 634, 635, 636, 637]
User 204,	 Attempt: 1 	 [686, 687]
User 219,	 Attempt: 1 	 [728, 729, 730, 732, 733, 734

In [18]:
print("Answers removed as attempt is not completed: \t\t\t\t{}".format(
    ans_removed_cause_incomplete))
print("Attempts removed as attempt is not completed: \t\t\t\t{}".format(
    att_removed_cause_incomplete))
print("Answers removed as answer is given faster than 60 seconds: \t{}".format(
    ans_removed_cause_fast))
print("Attempts removed as answer is given faster than 60 seconds: \t{}".format(
    att_removed_cause_fast))


Answers removed as attempt is not completed: 				82
Attempts removed as attempt is not completed: 				30
Answers removed as answer is given faster than 60 seconds: 	48
Attempts removed as answer is given faster than 60 seconds: 	6


In [19]:
# perform cleanup entries:

print("Before cleanup: {}".format(len(res)))
for r in toRemove:
    res.remove(r)
print("After cleanup: {}".format(len(res)))

if not os.path.exists(TARGET_DIR):
    os.makedirs(TARGET_DIR)

with open(os.path.join(TARGET_DIR, 'test_Results_cleaned+mintime+fullAtt.json'), 'w') as outfile:
    outfile.write(json.dumps(res, indent=4))


Before cleanup: 1146
After cleanup: 1016


In [20]:
# # Cleanup off survey data
# sessionIDs_to_remove = [x['sessionID'] for x in toRemove]
# sessionID_to_num_attempts = defaultdict(list)
# for x in toRemove:
#     sessionID_to_num_attempts[x['sessionID']].append(x["attempt"])
# to_remove_form_survey = []

# path_to_survey = '../study/rawresults/test_DB_Survey.json'

# with open(path_to_survey, "r") as jsonRes:
#     survey = json.load(jsonRes)
#     jsonRes.close()

#     for data in survey:
#         if data['sessionid'] in sessionIDs_to_remove \
#                 and len(sessionID_to_num_attempts[data['sessionid']]) <= 1:  # Do not remove the survey data if we delete a second, third attempt
#             to_remove_form_survey.append(data)

#     print(len(survey))
#     for d in to_remove_form_survey:
#         survey.remove(d)
#     print(len(survey))

#     with open('../data/test_survey_cleaned.json', 'w') as outfile:
#         outfile.write(json.dumps(survey, indent=4))


Load the data and merge them for later and easier processing


In [21]:

# Load the data

import numpy as np
import json
import javalang
from collections import defaultdict


path_to_results = os.path.join(TARGET_DIR, 'test_Results_cleaned+mintime+fullAtt.json')
path_to_survey = '../study/rawresults/test_DB_Survey.json'
path_to_oracle = '../study/benchmark.jsonl'

# Load the answers
with open(path_to_results, "r") as jsonRes:
    res = json.load(jsonRes)
    jsonRes.close()

# Load the survey data
with open(path_to_survey) as jsonSurveyFile:
    jsonSurvey = json.load(jsonSurveyFile)
    jsonSurveyFile.close()
session_id_to_survey = dict()
for survey in jsonSurvey:
    session_id_to_survey[survey["sessionid"]] = survey


# Load the oracle (correct answres)
with open(path_to_oracle, 'r') as json_file:
    oracle_list = list(json_file)
    json_file.close()
oracle = dict()
for i, json_str in enumerate(oracle_list):
    obj = json.loads(json_str)
    oracle[i] = obj


for answer in res:
    # Merge the data with the survey data
    survey_data = session_id_to_survey[answer["sessionID"]]
    if survey_data is not None:
        for key, value in survey_data.items():
            if key not in ["sessionid", "id"]:
                answer[key] = value
    else:
        print("Could not load the survey data for element {}".format(
            answer["sessionID"]))

    # On the interface, the line number started with 1. In the benchmark, the line number starts with 0.
    answer["lineNumberOfError"] -= 1

    # Merge with expected output
    # Load the expected answer (which is the line number)
    expected_answer = oracle[answer["ProgrammID"]]["line_num"]
    if expected_answer is not None:
        answer["expectedAnswer"] =  \
            - 1 if expected_answer < 1 else expected_answer

        exceptionIsTrue = np.sign([answer["expectedAnswer"]])[0] >= 1.0
        answerIsTrue = np.sign([answer["lineNumberOfError"]])[0] >= 1.0
        answer["taskSolvedCorrectly"] = (exceptionIsTrue is answerIsTrue)
        answer["correctLineNumber"] = answer["taskSolvedCorrectly"] and (
            answer["expectedAnswer"] == answer["lineNumberOfError"])
        answer['func_code'] = oracle[answer["ProgrammID"]]['func_code'],
        answer['experience'] = session_id_to_survey[answer["sessionID"]]['experience'],
        answer['age'] = session_id_to_survey[answer["sessionID"]]['age']
        answer['task'] = session_id_to_survey[answer["sessionID"]]['task']
    else:
        print("Could not load the oracle answer for element {}".format(
            answer["ID"]))


In [22]:
def tokenize(x):
    x = x + " ;"  # Sometimes this is necessary to perform the parse
    tokens = [t.value for t in javalang.tokenizer.tokenize(x)]
    return tokens[:-1]


In [23]:
results_by_user = defaultdict(list)
for answer in res:
    participant_id = answer['sessionID']
    # TODO: i don't know why this is needed but solves the problem of duplicated elements
    already_present = False
    for data in results_by_user[participant_id]:
        if data['ProgrammID'] == answer['ProgrammID']:
            already_present = True
    if not already_present:
        results_by_user[participant_id].append({
            "ProgrammID": answer['ProgrammID'],
            "lineNumberOfError": answer['lineNumberOfError'],
            "attempt": answer['attempt'],
            "expectedAnswer": answer['expectedAnswer'],
            "taskSolvedCorrectly": answer['taskSolvedCorrectly'],
            "correctLineNumber": answer['correctLineNumber'],
            "timestamp": answer['timestamp'],
            "program": answer['func_code'][0],
            "exp": answer['experience'][0],
            "age": answer['age'],
            "task": answer['task'],
            "token": len(tokenize(answer['func_code'][0])),
            "ctx": answer['ctx'],
            "codebase": answer['codebase'],
            "team": answer['team']
        })


In [24]:
with open(os.path.join(TARGET_DIR, 'answers_by_user.json'), 'w') as outfile:
    outfile.write(json.dumps(results_by_user, indent=4))


In [25]:
print("Non-buggy tasks",
      len([x for x in oracle.values() if x['line_num'] <= 0]))
print("Non-buggy tasks",
      len([x for x in oracle.values() if x['line_num'] > 0]))


Non-buggy tasks 176
Non-buggy tasks 134
